<a href="https://colab.research.google.com/github/Maplewarrior/RB-NLI/blob/main/RB_NLI_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Uncertainty NLI")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import RBOAA dependencies

In [ ]:
os.chdir("/content/drive/MyDrive/Uncertainty NLI/RBOAA")
from RBOAA_class import _RBOAA
os.chdir('/content/drive/MyDrive/Uncertainty NLI')

In [ ]:
chaosSNLI = pd.read_json(path_or_buf='/content/drive/MyDrive/Uncertainty NLI/data/chaosNLI_v1.0/chaosNLI_snli.jsonl', lines=True)
chaosMNLI = pd.read_json(path_or_buf='/content/drive/MyDrive/Uncertainty NLI/data/chaosNLI_v1.0/chaosNLI_mnli_m.jsonl', lines=True)
chaosANLI = pd.read_json(path_or_buf='/content/drive/MyDrive/Uncertainty NLI/data/chaosNLI_v1.0/chaosNLI_alphanli.jsonl', lines=True)


In [ ]:
chaosSNLI.head()
#print(chaosSNLI['example'][0])


,uid,label_counter,majority_label,label_dist,label_count,entropy,example,old_label,old_labels
0,2407214681.jpg#0r1n,"{'n': 70, 'e': 30}",n,"[0.30000000000000004, 0.7000000000000001, 0.0]","[30, 70, 0]",0.881291,"{'uid': '2407214681.jpg#0r1n', 'premise': 'Two...",n,"[neutral, neutral, neutral, entailment, entail..."
1,4718146904.jpg#2r1n,"{'n': 94, 'e': 3, 'c': 3}",n,"[0.03, 0.9400000000000001, 0.03]","[3, 94, 3]",0.387445,"{'uid': '4718146904.jpg#2r1n', 'premise': 'A m...",n,"[neutral, entailment, entailment, neutral, neu..."
2,3980085662.jpg#0r1e,"{'e': 98, 'n': 2}",e,"[0.98, 0.02, 0.0]","[98, 2, 0]",0.141441,"{'uid': '3980085662.jpg#0r1e', 'premise': 'Two...",e,"[entailment, neutral, entailment, entailment, ..."
3,24467676.jpg#1r1e,"{'e': 98, 'c': 2}",e,"[0.98, 0.0, 0.02]","[98, 0, 2]",0.141441,"{'uid': '24467676.jpg#1r1e', 'premise': 'A man...",e,"[entailment, neutral, neutral, entailment, ent..."
4,3667788497.jpg#0r1e,{'e': 100},e,"[1.0, 0.0, 0.0]","[100, 0, 0]",0.000000,"{'uid': '3667788497.jpg#0r1e', 'premise': 'Und...",e,"[entailment, neutral, entailment, neutral, ent..."


In [ ]:
def extractSingleAnntotations(df, n_annotators=100):
  # n = 1, e = 2, c = 3
  N = len(df)
  label_counts = df['label_counter']

  answer_dist = [{'n':0, 'e':0, 'c':0} for _ in range(N)]
  #annotations = [np.empty((n_annotators, 1)) for _ in range(N)]
  annotations = []
  for i in range(N):
    answer_dist[i]['n'] = label_counts[i]['n'] if 'n' in list(label_counts[i].keys()) else 0
    answer_dist[i]['e'] = label_counts[i]['e'] if 'e' in list(label_counts[i].keys()) else 0
    answer_dist[i]['c'] = label_counts[i]['c'] if 'c' in list(label_counts[i].keys()) else 0
    # baseline annotations method...
    annotations.append(np.expand_dims(np.repeat([1,2,3], [answer_dist[i]['n'], answer_dist[i]['e'], answer_dist[i]['c']]), axis=0))
  
  new_df = df[['example', 'entropy']].copy()
  new_df['answer_dist'] = answer_dist
  new_df['annotations'] = annotations
  return new_df


  
  
  

In [ ]:
label_counts = chaosSNLI['label_counter']
answer_dist = [{'n':0, 'e':0, 'c':0} for _ in range(len(label_counts))]
for i in range(len(label_counts)):
  answer_dist[i]['n'] = label_counts[i]['n'] if 'n' in list(label_counts[i].keys()) else 0
  answer_dist[i]['e'] = label_counts[i]['e'] if 'e' in list(label_counts[i].keys()) else 0
  answer_dist[i]['c'] = label_counts[i]['c'] if 'c' in list(label_counts[i].keys()) else 0

In [ ]:
df_s = extractSingleAnntotations(chaosSNLI) 
df_s.head()

,example,entropy,answer_dist,annotations
0,"{'uid': '2407214681.jpg#0r1n', 'premise': 'Two...",0.881291,"{'n': 70, 'e': 30, 'c': 0}","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1,"{'uid': '4718146904.jpg#2r1n', 'premise': 'A m...",0.387445,"{'n': 94, 'e': 3, 'c': 3}","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
2,"{'uid': '3980085662.jpg#0r1e', 'premise': 'Two...",0.141441,"{'n': 2, 'e': 98, 'c': 0}","[[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,..."
3,"{'uid': '24467676.jpg#1r1e', 'premise': 'A man...",0.141441,"{'n': 0, 'e': 98, 'c': 2}","[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,..."
4,"{'uid': '3667788497.jpg#0r1e', 'premise': 'Und...",0.000000,"{'n': 0, 'e': 100, 'c': 0}","[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,..."


In [ ]:
RBOAA = _RBOAA()
#df_s['annotations'][0].shape
res = RBOAA._compute_archetypes(X=df_s['annotations'][0], K=3, n_iter=1000, lr=0.01, mute=False, columns=['Q1'])


Response Bias Ordinal Arhcetypal Analysis in progress...
Finished Successfully after 2.9 seconds!                               

/////////////// INFORMATION ABOUT ORDINAL ARCHETYPAL ANALYSIS \\\\\\\\\\\\\\\\
▣ The Ordinal Archetypal Analysis was computed using 3 archetypes.
▣ The Ordinal Archetypal Analysis was computed on 1 attributes.
▣ The Ordinal Archetypal Analysis was computed on 100 subjects.
▣ The Ordinal Archetypal Analysis ran for 1000 itterations.
▣ The Ordinal Archetypal Analysis took 2.87 seconds to complete.
▣ The final loss was: 0.006264656316488981.


In [ ]:
res.Z.shape
res.A.shape

NameError: ignored

## Problems/questions

* We have no way of knowing the actual answers amongst respondents → How do we deal with this? 
* Using archetypal analysis might just loose information from the existing dataset... 
* How can we optimally use the information gathered by AA?

* Pivot? → Cluster groups of people based on their closest  archetype across examples?


## Appropriate datasets:
https://github.com/epavlick/NLI-variation-data/tree/master/context-analysis/raw

https://direct.mit.edu/tacl/article/doi/10.1162/tacl_a_00293/43531/Inherent-Disagreements-in-Human-Textual-Inferences?fbclid=IwAR0KokaN1zCGAl1x9Z3BhWCFaSL04WSs2eINdwqlvuz9lZd2tuKiLset61M


* UNLI dataset (Chen et al., 2020) only has two annotators (maybe 3) per example


In [ ]:
def entropy(a):
  return -sum([e*np.log(e) for e in a])

# A = archetype weighting matrix, n_subjects = how many subjects should be considered
def get_gold_answer(df, A, n_subjects):
  entropies = [entropy(a) for a in res.A.T]
  candidates = []
  for i in range(n_subjects):
    s = np.argmax(entropies)
    # i is added to account for the entropies list getting shorter
    candidates.append(s+i)
    entropies.pop(s)


  answers = [df['annotations'][candidates[i]] for i in range(n_subjects)]

  return candidates


  
ans = get_gold_answer(df_s, res.A, 10)

  

subject = np.argmax([entropy(a) for a in res.A.T])


NameError: ignored

In [ ]:
df_s['annotations'][0][0][0]
ans

[89, 91, 95, 87, 79, 44, 82, 37, 89, 27]

# $\textbf{NLI-variation dataset}$

In [ ]:
NLIcontext = pd.read_json(path_or_buf='/content/drive/MyDrive/Uncertainty NLI/data/NLI-variation-data/context-analysis/preprocessed-context-data.jsonl', lines=True)
NLIsent = pd.read_json(path_or_buf='/content/drive/MyDrive/Uncertainty NLI/data/NLI-variation-data/sentence-pair-analysis/preprocessed-data.jsonl', lines=True)

In [ ]:
print(NLIsent['premise'][0])
print(NLIsent['hypothesis'][0])
print(NLIsent['labels'][0])

What are they doing right that I did wrong? Maybe my mistake really was just a freak accident.
It is possible I was just a freak accident.
[49.999903547520425, -49.99995111441214, -49.9999262535316, -50.00002216690942, -49.99997016358147, 2.9743475205649e-05, -49.99991864902569, 50.000005487734626, 50.00007020674138, -37.999962284357004, 9.935426996016205e-05, -50.00001821982882, 23.000030071125416, 7.6247720714058375e-06, 50.00005718658829, -50.000053039841006, -49.99992127683262, -28.99996378762132, -46.00002935867597, 3.000081755211809, -49.999960995140164, -50.00005181855073, -28.000016451461523, 49.99999400284343, 2.7224880593486224e-05, 49.99998363923796, -50.00009330891493, 50.00009220806719, 23.999995182149863, -50.00009794576143, -34.000077596372115, -49.999965118989365, 50.00007936875679, -46.99994782046449, -49.99994262522547, -49.99992367330478, -49.999939137711976, -18.000055500136305, 14.999944639674295, -50.000014394180795, 4.9999653087219045, -50.00007802552399, -47.999

In [ ]:
context_raw1 = pd.read_csv('/content/drive/MyDrive/Uncertainty NLI/data/NLI-variation-data/context-analysis/raw/batch1.csv')
sent_raw1 = pd.read_csv('/content/drive/MyDrive/Uncertainty NLI/data/NLI-variation-data/sentence-pair-analysis/raw/batch1.csv')
sent_raw2 = pd.read_csv('/content/drive/MyDrive/Uncertainty NLI/data/NLI-variation-data/sentence-pair-analysis/raw/batch2.csv')
sent_raw3 = pd.read_csv('/content/drive/MyDrive/Uncertainty NLI/data/NLI-variation-data/sentence-pair-analysis/raw/batch3.csv')
attributeNames = list(context_raw1.columns)
attributeNames[:30]

['HITId',
 'HITTypeId',
 'Title',
 'Description',
 'Keywords',
 'Reward',
 'CreationTime',
 'MaxAssignments',
 'RequesterAnnotation',
 'AssignmentDurationInSeconds',
 'AutoApprovalDelayInSeconds',
 'Expiration',
 'NumberOfSimilarHITs',
 'LifetimeInSeconds',
 'AssignmentId',
 'WorkerId',
 'AssignmentStatus',
 'AcceptTime',
 'SubmitTime',
 'AutoApprovalTime',
 'ApprovalTime',
 'RejectionTime',
 'RequesterFeedback',
 'WorkTimeInSeconds',
 'LifetimeApprovalRate',
 'Last30DaysApprovalRate',
 'Last7DaysApprovalRate',
 'Input.task_name_0',
 'Input.pair_id_0',
 'Input.label_0']

In [ ]:

questions = list(pd.concat([context_raw1['HITId'], sent_raw1['HITId'], sent_raw2['HITId'], sent_raw3['HITId']]))
workers = list(pd.concat([context_raw1['WorkerId'], sent_raw1['WorkerId'], sent_raw2['WorkerId'], sent_raw3['WorkerId']], ignore_index=True))

n_workers = len(workers)
n_unique = len(set(workers))
n_questions = len(set(questions))

print(f'total workers: {n_workers}')
print(f'unique workers: {n_unique}')
print(f'unique questions: {n_questions}')


total workers: 2361
unique workers: 829
unique questions: 71


33.25352112676056

In [ ]:
N = len(context_raw1)

ws = list(context_raw1['WorkerId'])
qs = list(context_raw1['HITId'])

comb = context_raw1[['HITId', 'WorkerId']]



print(f"workers per question, context: {len(context_raw1['WorkerId'])/len(set(context_raw1['HITId']))} ")

print(f"workers per question, sent1: {len(set(sent_raw1['WorkerId']))/len(set(sent_raw1['HITId']))} ")
print(f"workers per question, sent2: {len(set(sent_raw2['WorkerId']))/len(set(sent_raw2['HITId']))} ")
print(f"workers per question, sent3: {len(set(sent_raw3['WorkerId']))/len(set(sent_raw3['HITId']))} ")

workers per question, context: 50.0 
workers per question, sent1: 11.833333333333334 
workers per question, sent2: 7.526315789473684 
workers per question, sent3: 12.28 


In [ ]:
set(sent_raw1['HITId'])


{'3D42WVSDH7U04HX7IZ3X5OTSDU8YFV',
 '3EKTG13IZT2MASIPL7IK66B51GGLMX',
 '3HXCEECSQLS69YO7JBX48G9APZ2ZYK',
 '3MYASTQBG6AUXVSP0JRD42N528BQD9',
 '3N2YPY1GI5XRU926IDW7G74YJCRVE1',
 '3ZFRE2BDQ8DH9TYXEFB2P221NT8ZXE'}